In [1]:
#Library for reading data as pandas dataframe
import pandas as p

In [20]:
#Libraries for the dashboard
import dash
import dash_core_components as dcc
import dash_html_components as html



In [2]:
from plotly import __version__
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
init_notebook_mode(connected=True)


# Loading of data

In [3]:
#Movement data on Saturday
data_sat=p.read_csv('C:/Users/Eugene/Desktop/Courses of the Master of Statistics/Fourth Semester/Data Visualization in data science/save-Sat.csv')
sat_count=data_sat.groupby(['type','Zone']).size().reset_index().rename(columns={0:'count'})

In [4]:
max(data_fri['id'])

2096715

In [44]:
#To be used for heatmap showing relationship between typpe of activity and location on saturday
sat_count_m=data_sat.groupby(['type','Zone']).size().reset_index().rename(columns={0:'count'})
sat_count_m

,type,Zone,count
0,check-in,Coaster Alley,23232
1,check-in,Entry Corridor,4064
2,check-in,Kiddie Land,10931
3,check-in,Tundra Land,17408
4,check-in,Wet Land,38441
5,movement,Coaster Alley,929518
6,movement,Entry Corridor,1854465
7,movement,Kiddie Land,625974
8,movement,Tundra Land,1373587
9,movement,Wet Land,1771481


In [29]:
#Communication data on Friday
data=p.read_csv('C:/Users/Eugene/Desktop/Courses of the Master of Statistics/Fourth Semester/Data Visualization in data science/comm-data-Fri.csv',
               parse_dates = ['Timestamp'])


In [30]:
#Communication data for Saturday
data2=p.read_csv('C:/Users/Eugene/Desktop/Courses of the Master of Statistics/Fourth Semester/Data Visualization in data science/comm-data-Sat.csv',
               parse_dates = ['Timestamp'])

In [31]:
#Communication data on Sunday
data3=p.read_csv('C:/Users/Eugene/Desktop/Courses of the Master of Statistics/Fourth Semester/Data Visualization in data science/comm-data-Sun.csv',
               parse_dates = ['Timestamp'])

# Generating matrices to use for heatmaps

In [9]:
data2.loc[data2['from']==1278894][0:4]

,Timestamp,from,to,location
343025,2014-06-07 12:00:00,1278894,1562191,Entry Corridor
343026,2014-06-07 12:00:00,1278894,211273,Entry Corridor
343027,2014-06-07 12:00:00,1278894,1023506,Entry Corridor
343028,2014-06-07 12:00:00,1278894,764970,Entry Corridor


In [32]:
#To use in Heatmap for communication from attraction sites

sat_count_comm=data2.groupby(['from','location']).size().reset_index().rename(columns={0:'count'})
sat_count_comm["location"] = p.Categorical(sat_count_comm["location"], sat_count_comm.location.unique())
sat_matrix = sat_count_comm.pivot("location", "from", "count")
sat_matrix=sat_matrix.fillna(0)

# Generating data frames to use for Line charts

In [33]:
#Collecting all unique ids into respective vectors
ind_RFE2=list(set(data_sat.loc[(data_sat['Site']=='Ride for everyone')&(data_sat['type']=='check-in')].id))
ind_KR2=list(set(data_sat.loc[(data_sat['Site']=='Kiddie rides')&(data_sat['type']=='check-in')].id))
ind_TR2=list(set(data_sat.loc[(data_sat['Site']=='Thrill rides')&(data_sat['type']=='check-in')].id))
ind_SE2=list(set(data_sat.loc[(data_sat['Site']=='Shows & Entertainment')&(data_sat['type']=='check-in')].id))
ind_BG2=list(set(data_sat.loc[(data_sat['Site']=='Beer Garden')&(data_sat['type']=='check-in')].id))
ind_S2=list(set(data_sat.loc[(data_sat['Site']=='Shopping')&(data_sat['type']=='check-in')].id))

In [34]:
ind_loc1=[ind_RFE2, ind_KR2, ind_TR2, ind_SE2, ind_BG2, ind_S2]
ind_loc2=[ind_RFE2, ind_KR2, ind_TR2, ind_SE2, ind_BG2, ind_S2]
com_vec=[]
for j in range(0,len(ind_loc1)):
    for i in range(0,len(ind_loc2)):
        m=set(ind_loc1[j])&set(ind_loc2[i])
        com_vec.append(len(m))
    
    
len(com_vec)
print(com_vec)

[6351, 4263, 6148, 2018, 4536, 2869, 4263, 4300, 4143, 1428, 2961, 1889, 6148, 4143, 6186, 1936, 4473, 2821, 2018, 1428, 1936, 2018, 2018, 867, 4536, 2961, 4473, 2018, 4555, 2114, 2869, 1889, 2821, 867, 2114, 2880]


In [35]:
#To use for line chart for saturday
#sat_count_comm3=data2.groupby(['Timestamp','from']).size().reset_index().rename(columns={0:'count'})
#fri_count_comm3 = fri_count_comm3[['count', 'from','TS_seconds']]
#Time_count_fri=fri_count_comm3.drop(['from'],axis=1)
#Time_count_sat=p.DataFrame({'Timestamp':sat_count_comm3['Timestamp'],
 #                          'count':sat_count_comm3['count']})
#sat_count_comm3=Time_count_sat.groupby('Timestamp').sum()
#sat_count_comm3.reset_index(inplace=True)
#sat_count_comm3[1:10]
#Time_count_fri[1:5]

In [36]:
#Group and summing number of messages sent on Friday per timestamp regardless of attraction site
#On Friday
fri_count=data.groupby(['Timestamp','from']).size().reset_index().rename(columns={0:'count'})
fri_count2=fri_count.drop(['from'],axis=1)
fri_count2=fri_count2.groupby('Timestamp').sum()
fri_count2.reset_index(inplace=True)

In [37]:
#On Saturday
sat_count=data2.groupby(['Timestamp','from']).size().reset_index().rename(columns={0:'count'})
sat_count2=sat_count.drop(['from'],axis=1)
sat_count2=sat_count2.groupby('Timestamp').sum()
sat_count2.reset_index(inplace=True)

In [38]:
# On Sunday data
sun_count=data3.groupby(['Timestamp','from']).size().reset_index().rename(columns={0:'count'})
sun_count2=sun_count.drop(['from'],axis=1)
sun_count2=sun_count2.groupby('Timestamp').sum()
sun_count2.reset_index(inplace=True)

In [39]:
#Count information per location
#Subset by location
data_TL=data2.loc[data2['location'] == 'Tundra Land']
data_WL=data2.loc[data2['location'] == 'Wet Land']
data_KL=data2.loc[data2['location'] == 'Kiddie Land']
data_EC=data2.loc[data2['location'] == 'Entry Corridor']
data_CA=data2.loc[data2['location'] == 'Coaster Alley']

In [40]:
#Group and summing number of messages sent per sender for each attraction site
sat_count_TL=data_TL.groupby(['Timestamp','from']).size().reset_index().rename(columns={0:'count'})
sat_count_TL=sat_count_TL.drop(['from'],axis=1)
sat_count_TL=sat_count_TL.groupby('Timestamp').sum()
sat_count_TL.reset_index(inplace=True)

sat_count_WL=data_WL.groupby(['Timestamp','from']).size().reset_index().rename(columns={0:'count'})
sat_count_WL=sat_count_WL.drop(['from'],axis=1)
sat_count_WL=sat_count_WL.groupby('Timestamp').sum()
sat_count_WL.reset_index(inplace=True)

sat_count_KL=data_KL.groupby(['Timestamp','from']).size().reset_index().rename(columns={0:'count'})
sat_count_KL=sat_count_KL.drop(['from'],axis=1)
sat_count_KL=sat_count_KL.groupby('Timestamp').sum()
sat_count_KL.reset_index(inplace=True)

sat_count_EC=data_EC.groupby(['Timestamp','from']).size().reset_index().rename(columns={0:'count'})
sat_count_EC=sat_count_EC.drop(['from'],axis=1)
sat_count_EC=sat_count_EC.groupby('Timestamp').sum()
sat_count_EC.reset_index(inplace=True)

sat_count_CA=data_CA.groupby(['Timestamp','from']).size().reset_index().rename(columns={0:'count'})
sat_count_CA=sat_count_CA.drop(['from'],axis=1)
sat_count_CA=sat_count_CA.groupby('Timestamp').sum()
sat_count_CA.reset_index(inplace=True)

# Building Application

In [ ]:
#Initializing the App
app = dash.Dash()
colors = dict(background= '#111111',text= '#0522FA')



#Building components of the app
app.layout = html.Div(
    children=[
    html.H1(children='AN APPLICATION BY EUGENE A. NDAMUKONG', style=dict(
        textAlign= 'center',
        color= colors['text']
    )
),

    html.Div(children='''
       SECTION A: VISUALIZATION OF THE MOVEMENT DATA
    '''),
    dcc.Graph(
        id='Common-Attractions',
        figure={
            'data': [
                #Heatmap(z=[[3517, 2395, 3490, 1176, 2707, 1829], [2395, 2414, 2393, 815, 1757, 1160],
          #[3490, 2393, 3502, 1170, 2703, 1828], [1176, 815, 1170, 1176, 1176, 585],[2707, 1757, 2703, 1176, 2709, 1434], [1829, 1160, 1828, 585, 1434, 1836]],
                        #x=['Ride for everyone','Kiddie rides','Thrill rides','Shows & Entertainment','Beer Garden','Shopping'],
                       # y=['Ride for everyone','Kiddie rides','Thrill rides','Shows & Entertainment','Beer Garden','Shopping'],colorscale=[[0.5,'#f7dcdc'],[0.8,'#ef5f62'],[1,'#f20408']]),
                Heatmap(z=[[6351, 4263, 6148, 2018, 4536, 2869], [4263, 4300, 4143, 1428, 2961, 1889], [6148, 4143, 6186, 1936, 4473, 2821],
          [2018, 1428, 1936, 2018, 2018, 867], [4536, 2961, 4473, 2018, 4555, 2114], [2869, 1889, 2821, 867, 2114, 2880]],
                        x=['Ride for everyone','Kiddie rides','Thrill rides','Shows & Entertainment','Beer Garden','Shopping'],
                        y=['Ride for everyone','Kiddie rides','Thrill rides','Shows & Entertainment','Beer Garden','Shopping'],colorscale=[[0.5,'#f7dcdc'],[0.8,'#ef5f62'],[1,'#f20408']])
                #Heatmap(z=[[7464, 4909, 7361, 982, 5026, 3390], [  4909, 4963, 4889, 559, 3059, 2216], [7361, 4889, 7432, 966, 5047, 3385],
          #[982, 559, 966, 982, 982, 420], [5026, 3059, 5047, 982, 5063, 2365], [3390, 2216, 3385, 420, 2365, 3419]],
                       #x=['Ride for everyone','Kiddie rides','Thrill rides','Shows & Entertainment','Beer Garden','Shopping'],
                       #y=['Ride for everyone','Kiddie rides','Thrill rides','Shows & Entertainment','Beer Garden','Shopping'],colorscale=[[0.5,'#f7dcdc'],[0.8,'#ef5f62'],[1,'#f20408']])
               
            ],
            'layout': Layout(
                xaxis={'title': 'Attraction'},
                yaxis={'title': 'Attraction'},
                title='Heatmap of Attraction with common visitors on Saturday',
                width=700,
                height=700,
                margin={'l': 150, 'b': 100, 't': 100, 'r': 50},
                #legend=False,
                hovermode='closest'
            )
        }
    
    ),
        #Line chart
     dcc.Graph(
        id='Type-Attraction-sites',
        figure={
            'data': [
                Heatmap(z=[sat_count_m['count'][i:i+5] for i  in range(0, len(sat_count_m['count']), 5)],
                        x=['Coaster Alley','Entry Corridor','Kiddie Land','Tundra Land', 'Wet Land'],
                        y=['check-in', 'movement'],colorscale=[[0,'#F5F5FA'],[.05,'#C8C9F7'],[1,'#4144FA']])
               
            ],
            'layout': Layout(
                xaxis={'title': 'Attraction sites'},
                yaxis={'title': 'Type of Activity'},
                title='Heatmap of Type of activity versus Attraction sites on Saturday',
                width=700,
                height=700,
                #margin={'l': 40, 'b': 40, 't': 10, 'r': 10},
                #legend={'x': 0, 'y': 1},
                hovermode='closest'
            )
        }
    
    ),
        html.Div(children='''
        SECTION B: VISUALIZATIONS OF THE COMMUNICATION DATA
    '''),
     dcc.Graph(
        id='Senders-Attraction',
        figure={
            'data': [
                Heatmap(z=sat_matrix.as_matrix(), x=sat_matrix.columns, y=sat_matrix.index,colorscale=[[0,'#F5F5FA'],[.05,'#5C5EF7'],[0.5,'#3928FA'],[1,'#000000']])
                
               
            ],
            'layout': Layout(
                xaxis={'title': 'Senders ID '},
                yaxis={'title': 'Attraction sites'},
                title='Heatmap of Communication from different attraction sites on Saturday',
                margin={'l': 100, 'b': 40, 't': 100, 'r': 50},
                width=1000,
                height=800,
                #legend={'x': 0, 'y': 1},
                hovermode='closest'
            )
        }
    
    ),
     dcc.Graph(
        id='Messaging-per-time-AllDays',
        figure={
            'data': [
                 Scatter(x = fri_count2['Timestamp'],y = fri_count2['count'],name = 'Messaging on Friday',mode='line'),
                 Scatter(x = sat_count2['Timestamp'],y = sat_count2['count'],name = 'Messaging on Saturday', mode='line'),
                 Scatter(x = sun_count2['Timestamp'],y = sun_count2['count'],name = 'Messaging on Sunday', mode='line')
            ],
            'layout': Layout(
                #xaxis={'title': 'Timestamp'},
                xaxis=dict(
                    rangeselector=dict(
                        buttons=list([
                            dict(count=6,
                                 label='6H',
                                 step='hour',
                                 stepmode='backward'),
                            dict(count=12,
                                 label='12H',
                                 step='hour',
                                 stepmode='backward'),
                            dict(count=60,
                                 label='1M',
                                 step='minute',
                                 stepmode='backward'),
                            dict(count=60,
                                 label='1S',
                                 step='second',
                                 stepmode='backward'),
                            dict(step='all')
                        ])
                    ),
                    rangeslider=dict(),type='date'
                ),
                yaxis=dict(title= 'Frequency of messaging'),
                title='Frequency of messaging on all Three days',
                height=400,
                margin={'l': 100, 'b': 40, 't': 100, 'r': 50},
                #legend={'x': 0, 'y': 1},
                hovermode='closest')
        }
    
    ),
        
     dcc.Graph(
        id='Messaging-per-time',
        figure={
            'data': [
                 Scatter(x = sat_count_TL['Timestamp'],y = sat_count_TL['count'],name = 'Messaging from Tundra Land',mode='line'),
                 Scatter(x = sat_count_WL['Timestamp'],y = sat_count_WL['count'],name = 'Messaging from Wet Land', mode='line'),
                 Scatter(x = sat_count_KL['Timestamp'],y = sat_count_KL['count'],name = 'Messaging from Kiddie Land', mode='line'),
                 Scatter(x = sat_count_EC['Timestamp'],y = sat_count_EC['count'],name = 'Messaging from Entrance Corridor',mode='line'),
                 Scatter(x = sat_count_TL['Timestamp'],y = sat_count_TL['count'],name = 'Messaging from Coaster Alley',mode='line')
            ],
            'layout': Layout(
                #xaxis={'title': 'Timestamp'},
                xaxis=dict(
                    rangeselector=dict(
                        buttons=list([
                            dict(count=6,
                                 label='6H',
                                 step='hour',
                                 stepmode='backward'),
                            dict(count=12,
                                 label='12H',
                                 step='hour',
                                 stepmode='backward'),
                            dict(count=60,
                                 label='1M',
                                 step='minute',
                                 stepmode='backward'),
                            dict(count=60,
                                 label='1S',
                                 step='second',
                                 stepmode='backward'),
                            dict(step='all')
                        ])
                    ),
                    rangeslider=dict(),type='date'
                ),
                yaxis=dict(title= 'Frequency of messaging'),
                title='Frequency of messaging on Saturday',
                margin={'l': 100, 'b': 40, 't': 100, 'r': 50},
                #legend={'x': 0, 'y': 1},
                hovermode='closest')
        }
    
    )
])

if __name__ == '__main__':
    app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [21/May/2018 20:35:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2018 20:35:08] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2018 20:35:21] "GET /_dash-layout HTTP/1.1" 200 -


In [34]:
app = dash.Dash()
app.layout = html.Div([

    html.Div([
        dcc.Graph(
            id='Common-Attractions',
            figure={
            'data': [
                Heatmap(z=[[6351, 4263, 6148, 2018, 4536, 2869], [4263, 4300, 4143, 1428, 2961, 1889], [6148, 4143, 6186, 1936, 4473, 2821],
          [2018, 1428, 1936, 2018, 2018, 867], [4536, 2961, 4473, 2018, 4555, 2114], [2869, 1889, 2821, 867, 2114, 2880]],
                        x=['Ride for everyone','Kiddie rides','Thrill rides','Shows & Entertainment','Beer Garden','Shopping'],
                        y=['Ride for everyone','Kiddie rides','Thrill rides','Shows & Entertainment','Beer Garden','Shopping'],colorscale=[[0.5,'#f7dcdc'],[0.8,'#ef5f62'],[1,'#f20408']])
               
            ],
            'layout': Layout(
                xaxis={'title': 'Attraction'},
                yaxis={'title': 'Attraction'},
                title='Heatmap of Attraction with common visitors on Saturday',
                height= 600,
                #margin={'l': 40, 'b': 40, 't': 10, 'r': 10},
                #legend={'x': 0, 'y': 1},
                hovermode='closest'
            )
        }
    
        ),
        dcc.Graph(
            id='Common-Attractions',
            figure={
            'data': [
                Heatmap(z=[sat_count['count'][i:i+5] for i  in range(0, len(sat_count['count']), 5)],
                        x=['Coaster Alley','Entry Corridor','Kiddie Land','Tundra Land', 'Wet Land'],
                        y=['check-in', 'movement'],colorscale=[[0,'#F5F5FA'],[.5,'#C8C9F7'],[1,'#4144FA']])
               
            ],
            'layout': Layout(
                xaxis={'title': 'Attraction sites'},
                yaxis={'title': 'Type of activities'},
                title='Heatmap of type of activities in the Attraction sites on Saturday',
                height=600,
                #margin={'l': 40, 'b': 40, 't': 10, 'r': 10},
                #legend={'x': 0, 'y': 1},
                hovermode='closest'
            )
        }
    
        )
    ], style={'width': '49%', 'display': 'inline-block','vertical-align': 'middle'}),
    html.Div([
        dcc.Graph(
        id='Senders-Attraction',
        figure={
            'data': [
                Heatmap(z=sat_matrix.as_matrix(), x=sat_matrix.columns, y=sat_matrix.index,colorscale=[[0,'#F5F5FA'],[.05,'#C8C9F7'],[1,'#4144FA']])
               
            ],
            'layout': Layout(
                xaxis={'title': 'Senders'},
                yaxis={'title': 'Attraction sites'},
                title='Heatmap of Communication from different attraction sites on Saturday',
                height= 400,
                #margin={'l': 100, 'b': 40, 't': 100, 'r': 10},
                #legend={'x': 0, 'y': 1},
                hovermode='closest'
            )
        }
    
    ),
       dcc.Graph(
        id='Messaging-per-time_saturday',
        figure={
            'data': [
                 Scatter(x = sat_count_TL['Timestamp'],y = sat_count_TL['count'],name = 'Messaging from Tundra Land',mode='line'),
                 Scatter(x = sat_count_WL['Timestamp'],y = sat_count_WL['count'],name = 'Messaging from Wet Land', mode='line'),
                 Scatter(x = sat_count_KL['Timestamp'],y = sat_count_KL['count'],name = 'Messaging from Kiddie Land', mode='line'),
                 Scatter(x = sat_count_EC['Timestamp'],y = sat_count_EC['count'],name = 'Messaging from Entrance Corridor',mode='line'),
                 Scatter(x = sat_count_TL['Timestamp'],y = sat_count_TL['count'],name = 'Messaging from Coaster Alley',mode='line')
            ],
            'layout': Layout(
                #xaxis={'title': 'Timestamp'},
                xaxis=dict(
                    rangeselector=dict(
                        buttons=list([
                            dict(count=6,
                                 label='6H',
                                 step='hour',
                                 stepmode='backward'),
                            dict(count=12,
                                 label='12H',
                                 step='hour',
                                 stepmode='backward'),
                            dict(count=60,
                                 label='30M',
                                 step='minute',
                                 stepmode='backward'),
                            dict(count=60,
                                 label='1S',
                                 step='second',
                                 stepmode='backward'),
                            dict(step='all')
                        ])
                    ),
                    rangeslider=dict(),type='date'
                ),
                yaxis=dict(title= 'Frequency of messaging'),
                title='Frequency of messaging on Saturday',
                height=400,
                #margin={'l': 40, 'b': 40, 't': 10, 'r': 10},
                #legend={'x': 0, 'y': 1},
                hovermode='closest')
        }
    
    ),
        dcc.Graph(
        id='Messaging-per-time-AllDays',
        figure={
            'data': [
                 Scatter(x = fri_count2['Timestamp'],y = fri_count2['count'],name = 'Messaging on Friday',mode='line'),
                 Scatter(x = sat_count2['Timestamp'],y = sat_count2['count'],name = 'Messaging on Saturday', mode='line'),
                 Scatter(x = sun_count2['Timestamp'],y = sun_count2['count'],name = 'Messaging on Sunday', mode='line')
            ],
            'layout': Layout(
                #xaxis={'title': 'Timestamp'},
                xaxis=dict(
                    rangeselector=dict(
                        buttons=list([
                            dict(count=6,
                                 label='6H',
                                 step='hour',
                                 stepmode='backward'),
                            dict(count=12,
                                 label='12H',
                                 step='hour',
                                 stepmode='backward'),
                            dict(count=60,
                                 label='1M',
                                 step='minute',
                                 stepmode='backward'),
                            dict(count=60,
                                 label='1S',
                                 step='second',
                                 stepmode='backward'),
                            dict(step='all')
                        ])
                    ),
                    rangeslider=dict(),type='date'
                ),
                yaxis=dict(title= 'Frequency of messaging'),
                title='Frequency of messaging on all Three days',
                height=400,
                #margin={'l': 40, 'b': 40, 't': 10, 'r': 10},
                #legend={'x': 0, 'y': 1},
                hovermode='closest')
        }
    
    )
    ], style= {'width': '49%', 'display': 'inline-block', 'vertical-align': 'middle'})
])

if __name__ == '__main__':
    app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/May/2018 23:37:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2018 23:37:12] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2018 23:37:33] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2018 23:53:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2018 23:54:00] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2018 23:54:48] "GET /_dash-layout HTTP/1.1" 200 -
